In [1]:
import database
import random
from event_generator import event_generator

In [17]:
db = database.DataBase()
priority_list = db.get_record('event_types',columns_to_select={'entityid','priority'},all_rows=True)
db.close()
priority_list = {x[0]:x[1] for x in priority_list}

ProgrammingError: 1054 (42S22): Unknown column 'entityid' in 'field list'

In [10]:
def find_collisions(events):
    collisions = []
    try:
        sorted_events = sorted(events,key=lambda event:(event[2],priority_list[event[1]]))
    except TypeError:
        print(f'TypeError on {sample_id = }')
        return -1
    events_by_time = {}
    for event in sorted_events:
        time = event[2]
        if events_by_time.get(time):
            events_by_time[time].append(event[1])
        else:
            events_by_time[time] = [event[1]]

    for occurred_types in events_by_time.values():
        if len(occurred_types) == 1 or occurred_types in collisions:
            continue
        seen_priority = []
        for event_type in occurred_types:
            if priority_list[event_type] in seen_priority:    
                collisions.append(occurred_types)
                break
            else:
                seen_priority.append(priority_list[event_type])
    return collisions

def remove_resolved_collisions(collisions):
    resolved_collisions = [[6,6],[44,44],[32,32],[30,30],[34, 34, 32, 32]]
    for resolved_collision in resolved_collisions:
        if resolved_collision in collisions:
            collisions.remove(resolved_collision)

def event_getter(events):
    for event in events:
        yield event

In [19]:
collisions = []
db = database.DataBase()
db.crs.execute('SELECT DISTINCT globalGameId FROM event_meta')
game_ids = [x[0] for x in db.crs.fetchall()]
sample_ids = random.sample(game_ids,50)
for sample_id in sample_ids:
    events = db.get_record('event_meta',columns_to_select={'globalEventId','type','expandedTime'},conditions={'globalGameId':sample_id},all_rows=True)
    new_collisions = find_collisions(events)
    if new_collisions == -1:
        db.close()
        quit()
    else:
        collisions += new_collisions
remove_resolved_collisions(collisions)

In [4]:
db = database.DataBase()
db.crs.execute('SELECT DISTINCT globalGameId FROM event_meta')
game_ids = [x[0] for x in db.crs.fetchall()]
sample_id = random.choice(game_ids)
print(f'{sample_id = }')
events = db.get_record('event_meta',columns_to_select={'globalEventId','type','expandedTime'},conditions={'globalGameId':sample_id},all_rows=True)
db.close()

sample_id = 1376043


In [11]:
collisions = find_collisions(events)

NameError: name 'x' is not defined

In [9]:
print(priority_list)

{1: 'Pass', 2: 'OffsidePass', 3: 'TakeOn', 4: 'Foul', 6: 'CornerAwarded', 7: 'Tackle', 8: 'Interception', 10: 'Save', 11: 'Claim', 12: 'Clearance', 13: 'MissedShots', 14: 'ShotOnPost', 15: 'SavedShot', 16: 'Goal', 17: 'Card', 18: 'SubstitutionOff', 19: 'SubstitutionOn', 30: 'End', 32: 'Start', 34: 'FormationSet', 40: 'FormationChange', 41: 'Punch', 42: 'GoodSkill', 44: 'Aerial', 45: 'Challenge', 49: 'BallRecovery', 50: 'Dispossessed', 51: 'Error', 52: 'KeeperPickup', 53: 'CrossNotClaimed', 54: 'Smother', 55: 'OffsideProvoked', 56: 'ShieldBallOpp', 58: 'PenaltyFaced', 59: 'KeeperSweeper', 60: 'ChanceMissed', 61: 'BallTouch', 74: 'BlockedPass', 10000: 'OffsideGiven'}
